# Segmentace obrazu - text a jeho čtení
Cvičení je zaměřené na správné využití osvětlení při nasvícení objektu a následné využití metod pro segmentaci obrazu. Segmentovaný snímek je převeden na text pomocí OCR (optical character recognition).

<img src="images/ocr.png" width="300">

Ke čtení je využita knihovna [**Tesseract OCR**](https://github.com/tesseract-ocr/tesseract).

### Import knihoven a konfigurace

In [ ]:
%run ./library.ipynb

### Pomocné funkce
Z následujících funkcí je potřeba vybírat ty vhodné pro splnění úkolu. Parametry a implementaci funkcí si můžete zobrazit pomocí příkazu `help(function_name)` nebo na https://gitlab.fit.cvut.cz/bi-svz/improutils_package/tree/master/improutils.

Seznam funkcí pro přehlednost:

- [`to_gray(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/preprocessing/preprocessing.py#L4)
- [`negative(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/preprocessing/preprocessing.py#L53)
- [`crop(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/preprocessing/preprocessing.py#L99)

- [`rotate(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/preprocessing/preprocessing.py#L184)

- [`segmentation_one_threshold(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/-/blob/master/improutils/segmentation/segmentation.py#L56)
- [`segmentation_auto_threshold(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/-/blob/master/improutils/segmentation/segmentation.py#L74)
- [`segmentation_two_thresholds(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/-/blob/master/improutils/segmentation/segmentation.py#L90)
- [`segmentation_adaptive_threshold(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/-/blob/master/improutils/segmentation/segmentation.py#L109), vysvětlení [použití](https://courses.fit.cvut.cz/BI-SVZ/showroom/adaptive_thresholding/index.html)

- [`find_contours(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/-/blob/master/improutils/preprocessing/contours.py#L40)
- [`fill_holes(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/-/blob/master/improutils/preprocessing/contours.py#L74)

- [`filtration_median(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/-/blob/master/improutils/filtration/filtration.py#L22)

- [`ocr(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/-/blob/master/improutils/recognition/ocr.py#L7)

- [`cv2.morphologyEx(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/), vysvětlení [použití](https://courses.fit.cvut.cz/BI-SVZ/showroom/morfologie/index.html)
- [`cv2.dilate(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/), vysvětlení [použití](https://courses.fit.cvut.cz/BI-SVZ/showroom/morfologie/index.html)
- [`cv2.erode(...)`](https://gitlab.fit.cvut.cz/bi-svz/improutils_package/blob/master/improutils/), vysvětlení [použití](https://courses.fit.cvut.cz/BI-SVZ/showroom/morfologie/index.html)

- `find_holes(...)`, nachází se v souboru *library.ipynb*
- template pro `interact slider`, jenž můžete (ale nemusíte) použít při segmentaci, je také v souboru *library.ipynb*

---

### Úkoly

1. Změřte velikost placky s nápisem FIT v pixelech - výška, šířka.

1. Zvolte vhodné funkce pro segmentaci obrazu a přečtěte text z obrázku snímaném kamerou. Po vyladění algoritmu, využijte předzpracovaný obrázek a nechte ho přečíst OCR. V případě, že výsledek bude po porovnání `True`, úkol jste splnili.

Pro každý z úkolů je nutné zvolit vhodný typ osvětlení a následně placku nasnímat monochromatickou kamerou. Na každý úkol bude v ideálním případě potřeba jiný snímek. Volba vhodného nasvícení usnadní algoritmickou část úkolu. Při špatné volbě osvětlení nebude úkol uznán!

#### 1) Nasnímejte placky s nápisem FIT a zobrazte.
Snímky uložte do jedné složky. Použijte vhodné osvětlení!

In [ ]:
path = ... ### nezapomeňte cestu zakončit '/'
files = os.listdir(path)

images = []

for f in files:
    image = load_image(path + f) ### načtěte obrázek
    images.append(image)
        
print('Celkem nalezených obrázků: ' + str(len(images)))
plot_images(*images) #zobrazte snímky

#### 2) Změřte rozměry placky v pixelech

Úkolem je ze snímku segmentovat placku a změřit její rozměry - výška, šířka v pixelech.

Rozměry jsou vykresleny do původního snímku viz. obrázek. Pro pozicování textu a obrysu placky využijte znalost kontury, ze které byly hodnoty vypočítány.

<img src="images/measures_ref.png" width="300">

In [ ]:
def measurement_algorithm(img):
    
    ### algoritmus segmentace a měření placky
    
    ...
    
    height, width = ... ###
    sizes_drawn = img.copy()
    points = ... ###

    cv2.line(sizes_drawn, points[0], points[1], color=(255, 0, 0), thickness=3)
    cv2.putText(sizes_drawn, "{:.2f} px".format(width), points[0], 0, 1.5, (0, 0, 255), 2)
    
    cv2.line(sizes_drawn, points[1], points[2], color=(255, 0, 0), thickness=3)
    cv2.putText(sizes_drawn, "{:.2f} px".format(height), points[2], 0, 1.5, (0, 0, 255), 2)

    return mask, sizes_drawn

In [ ]:
img_measures = images[...]  ### doplňte index
mask, sizes_drawn = measurement_algorithm(img_measures)
plot_images(mask, sizes_drawn)

#### 3) Doplňte algoritmus pro segmentaci textu z nasnímaných placek.
Výstupem algoritmu by měl být binární obraz, kde pozadí má černou barvu a objekty (popředí) bílou. Binární obrázek zobrazte.

In [ ]:
def segmentation_algorithm(img):
    
    ### algoritmus segmentace textu 
   
    ...
    
    img = crop(...)
    
    ...
    
    return img_bin

In [ ]:
img_text = images[...] ### doplňte index
img_bin = segmentation_algorithm(img_text)
plot_images(img_bin)

#### 4) Využijte funkční algoritmus na snímky.
**POZOR:** Dokud nebude vstupní obraz vypadat následovně, **NEPOUŠTĚJTE** se do další části. 

<img src="images/fit_preprocessed.png" width="200">

#### 5) Využijte OCR na výsledný binární obrázek a zkontrolujte správnost.
**Hint:** Pokud se požadovaný výstup OCR neshoduje s referenčním textem, zařiďte aby nebyl text na obrázku nijak rotován a kolem textu byl alespoň 20px rámeček. Důvodem je použití Tesseract OCR, jenž je na dané věci velice náchylný.

In [ ]:
text = ...(img_bin) ###
ref_text = 'FIT'

print('Přečtený text je: ' + "'" + text.strip() + "'\n")
if text.strip() == ref_text:
    print('-> Úkol jste splnili!\n')
else:
    print('-> Úkol je třeba dál ladit ...')    
print('')


### 6) Bonusová část - změřte rozměry kostičky v milimetrech

Změřte a zobrazte rozměry placky v milimetrech, když víte že výška písmena I je 18 mm.